<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/Alignment/Projection_Work/fine_tuned_awesome_aligner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using the fine tuned  model in Awesome Aligner**

In [1]:
!pip install transformers==3.1.0
import torch
import transformers
import itertools

     |████████████████████████████████| 884 kB 9.3 MB/s 
     |████████████████████████████████| 895 kB 51.8 MB/s 
     |████████████████████████████████| 3.0 MB 12.8 MB/s 
     |████████████████████████████████| 1.2 MB 45.3 MB/s 


Get and preprocess the parallel data

In [2]:
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/nnwale.en
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/nnwale.ig

--2022-03-17 21:41:30--  https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/nnwale.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14382 (14K) [text/plain]
Saving to: ‘nnwale.en’

nnwale.en           100%[===================>]  14.04K  --.-KB/s    in 0s      

2022-03-17 21:41:31 (68.9 MB/s) - ‘nnwale.en’ saved [14382/14382]

--2022-03-17 21:41:31--  https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/nnwale.ig
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16403 (16K) [text/plain]
Saving to:

In [3]:
# opening the file in read mode
en_file = open("/content/nnwale.en", "r")

# reading the file
ensents = en_file.read()

# replacing end splitting the text
# when newline ('\n') is seen.
ensents = ensents.split("\n")
# print(ensents)
en_file.close()

In [4]:
# opening the file in read mode
ig_file = open("/content/nnwale.ig", "r")

# reading the file
igsents = ig_file.read()

# replacing end splitting the text
# when newline ('\n') is seen.
igsents = igsents.split("\n")
# print(igsents)
ig_file.close()

In [5]:
# Python zip() function can be used to map the lists altogether 
# to create a list of tuples using the command:list(zip(list))
lst1 = ensents
lst2 = igsents
en_ig_sents = list(zip(lst1,lst2))
print(en_ig_sents)

[('Pius Adesanmi To Be Buried On Saturday', "Aga- eme elimozu Pius Adesanmi n'ụbọchị Satọdee"), ('Adesanmi, born February 27, 1972, was author of the popular book Naija No Dey Carry Last, a 2015 collection of satirical essays.', "Adesanmi, nke amụrụ n'ọnwa February 27, 1972, bụ onye dere akwụkwọ amaara nke ọma bụ Naija No Dey Carry Last, mkpokọta ederede nkatọ nke afọ 2015 "), ('The family of late Prof Pius Adesanmi has announced the burial of the Nigerian-born Canadian scholar, writer, literary critic and columnist, who died on March 10, 2019 when an Ethiopian Airline aircraft crashed shortly after take-off.', "Ezinaụlọ Prof Pius Adesanmi nwụrụ anwụ ekwupụtala maka elimozu nwa afọ Naịjirịa nke gụrụ akwụkwọ Kanada, ode akwụkwọ, onye edemede, nke nwụrụ n'abalị nke iri n'ọnwa Maachị n'afọ 2019 oge Ụgbọelu nke Etiopịa gbariri ka nwa oge o fepụrụ gasịrị."), ('Adesanmi, born February 27, 1972, was author of the popular book Naija No Dey Carry Last, a 2015 collection of satirical essays.', "

Mount the drive, Load the multilingual BERT model and its tokenizer.

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
model = transformers.BertModel.from_pretrained('/content/gdrive/MyDrive/en_ig_alignout')
#model = '/content/gdrive/MyDrive/en_ig_alignout'

Some weights of BertModel were not initialized from the model checkpoint at /content/gdrive/MyDrive/en_ig_alignout and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Run the model and print the resulting alignments.

In [11]:
# pre-processing
for src,tgt in en_ig_sents:
  sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
  token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
  wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
  ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
  sub2word_map_src = []
  for i, word_list in enumerate(token_src):
    sub2word_map_src += [i for x in word_list]
  sub2word_map_tgt = []
  for i, word_list in enumerate(token_tgt):
    sub2word_map_tgt += [i for x in word_list]

  # alignment
  align_layer = 8
  threshold = 1e-3
  model.eval()
  with torch.no_grad():
    out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
    out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

    dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

    softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
    softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

    softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

  align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
  align_words = set()
  for i, j in align_subwords:
    align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

  # printing
  class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

  for i, j in sorted(align_words):
    print(f'{sent_src[i]},{sent_tgt[j]}')

Pius,Pius
Adesanmi,Adesanmi
Buried,elimozu
On,n'ụbọchị
Saturday,Satọdee
Adesanmi,,Adesanmi,
born,nke
February,February
27,,27,
1972,,1972,
was,bụ
of,n'ọnwa
the,onye
popular,dere
book,bụ
Naija,Naija
No,No
Dey,Dey
Carry,Carry
Last,,Last,
a,mkpokọta
2015,2015
collection,mkpokọta
collection,ederede
collection,nkatọ
satirical,ederede
satirical,nke
essays.,afọ
The,Ezinaụlọ
Prof,Prof
Pius,Pius
Adesanmi,Adesanmi
has,ekwupụtala
announced,ekwupụtala
the,elimozu
the,nwa
burial,elimozu
burial,afọ
of,nwa
of,afọ
Nigerian-born,Naịjirịa
Canadian,Kanada,
writer,,akwụkwọ,
literary,edemede,
critic,akwụkwọ,
columnist,,edemede,
who,nke
died,nwụrụ
on,n'ọnwa
March,Maachị
10,,Maachị
10,,n'afọ
2019,2019
when,oge
an,nke
Ethiopian,Etiopịa
shortly,nwa
after,fepụrụ
take-off.,gasịrị.
Adesanmi,,Adesanmi,
born,nke
February,February
27,,27,
1972,,1972,
was,bụ
of,n'ọnwa
the,onye
popular,dere
book,bụ
Naija,Naija
No,No
Dey,Dey
Carry,Carry
Last,,Last,
a,mkpokọta
2015,2015
collection,mkpokọta
collection,ederede
collection,